<a href="https://colab.research.google.com/github/beyzoskaya/deepAllergen/blob/main/deepAllergen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece tensorflow torch

In [52]:
!pip install biopython

In [53]:
!apt-get update
!apt-get install -y cd-hit

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,137 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [57]:
from Bio import SeqIO

input_files = ["all.train.fasta", "all.test.fasta"]

records = []
seen_sequences = set()

for file in input_files:
    for rec in SeqIO.parse(file, "fasta"):
        seq = str(rec.seq)
        if seq not in seen_sequences:
            seen_sequences.add(seq)
            records.append(rec)

SeqIO.write(records, "all_merged_nr_exact.fasta", "fasta")

print("Total unique sequences:", len(records))

Total unique sequences: 7125


In [58]:
!cd-hit -i all_merged_nr_exact.fasta -o all_clusters.fasta -c 0.5 -n 3

Program: CD-HIT, V4.8.1 (+OpenMP), Aug 20 2021, 08:39:56
Command: cd-hit -i all_merged_nr_exact.fasta -o
         all_clusters.fasta -c 0.5 -n 3

Started: Sun Feb 15 17:54:44 2026
                            Output                              
----------------------------------------------------------------
Discarding invalid sequence or sequence without identifier and description!

>allergen_Triosephos
MANQRKFFVGGNWKMNGDRAGIDSIISFMKGPLSADTEVVVGCPQCYLMYTREHLPSNIG
VAAQNCYKVAKGAFTGEISPSMIKDCGCEWVILGHSERRNVFNEPDTLISEKVGHALEAG
LKVIPCIGEKLEERESNRTEEVVFAQMKALVPNISDWSRVVIAYEPVWAIGTGKTATPEQ
AQEVHAKLRQWLRDNVNAEVADSTRIIYGGSVTPGNCKELAKTGDIDGFLVGGASLKPDF
VQIINARD>allergen_TriosephosphateIsomerase10MANQRKFFVGGNWKMN
GDKAAIDGIISFMKGPLNADTEVVVGCPQCYLMYTREHMPANIGVAAQNCYKTAKGAFTG
EISPAMIKDCGCEWVILGHSERRNVFGEPDQLISEKVGHALEAGLKVIPCIGEKLEERES
NRTEEVVFAQMKALVPNISDWSRVVIAYEPVWAIGTGKTATPEQAQDVHAKLRQWLRDNV
SPQVAESTRIIYGGSVSAGNCKELAKTGDIDGFLVGGASLKPDFVTIINARA
total seq: 7124
longest and shortest : 1388 and 11


In [59]:
import random
from Bio import SeqIO
from collections import defaultdict

random.seed(42)

CLUSTER_FILE = "all_clusters.fasta.clstr"
FASTA_FILE = "all_merged_nr_exact.fasta"

TRAIN_RATIO = 0.70
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# -----------------------------
# Parse clusters
# -----------------------------
clusters = defaultdict(list)
cluster_id = None

with open(CLUSTER_FILE) as f:
    for line in f:
        line = line.strip()

        if line.startswith(">Cluster"):
            cluster_id = line
        else:
            seq_id = line.split(">")[1].split("...")[0]
            clusters[cluster_id].append(seq_id)

print("Total clusters:", len(clusters))

# -----------------------------
# Load sequences
# -----------------------------
seq_dict = {rec.id: rec for rec in SeqIO.parse(FASTA_FILE, "fasta")}

# -----------------------------
# Shuffle clusters
# -----------------------------
cluster_list = list(clusters.values())
random.shuffle(cluster_list)

# -----------------------------
# Assign clusters
# -----------------------------
total_sequences = sum(len(c) for c in cluster_list)

train_ids = set()
val_ids = set()
test_ids = set()

count_train = 0
count_val = 0

for cluster in cluster_list:

    if count_train / total_sequences < TRAIN_RATIO:
        train_ids.update(cluster)
        count_train += len(cluster)

    elif count_val / total_sequences < VAL_RATIO:
        val_ids.update(cluster)
        count_val += len(cluster)

    else:
        test_ids.update(cluster)

# -----------------------------
# Save FASTA
# -----------------------------
def save_split(ids, filename):
    records = [seq_dict[i] for i in ids if i in seq_dict]
    SeqIO.write(records, filename, "fasta")
    print(f"{filename}: {len(records)} sequences")

save_split(train_ids, "train_safe.fasta")
save_split(val_ids, "val_safe.fasta")
save_split(test_ids, "test_safe.fasta")

# -----------------------------
# Leakage check
# -----------------------------
print("\nLeakage Check")
print("Train ∩ Val:", len(train_ids & val_ids))
print("Train ∩ Test:", len(train_ids & test_ids))
print("Val ∩ Test:", len(val_ids & test_ids))

Total clusters: 4337
train_safe.fasta: 5021 sequences
val_safe.fasta: 1069 sequences
test_safe.fasta: 1020 sequences

Leakage Check
Train ∩ Val: 0
Train ∩ Test: 0
Val ∩ Test: 0


In [60]:
from Bio import SeqIO

for f in ["train_safe.fasta","val_safe.fasta","test_safe.fasta"]:
    count = sum(1 for _ in SeqIO.parse(f, "fasta"))
    print(f"{f}: {count}")

train_safe.fasta: 5021
val_safe.fasta: 1069
test_safe.fasta: 1020


In [6]:
!python datapreprocess.py

2026-02-15 15:18:13.003097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771168693.022952    1527 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771168693.029712    1527 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771168693.045258    1527 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771168693.045283    1527 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771168693.045287    1527 computation_placer.cc:177] computation placer alr

In [34]:
import os

with open("train.py", "r") as f:
    content = f.read()

content = content.replace("import tensorflow_addons as tfa", "# import tensorflow_addons as tfa")

content = content.replace("tfa.optimizers.AdamW", "tf.keras.optimizers.AdamW")

with open("train.py", "w") as f:
    f.write(content)

print("✅ train.py has been patched to run without tensorflow-addons!")

✅ train.py has been patched to run without tensorflow-addons!


In [35]:
import os

with open("model.py", "r") as f:
    content = f.read()

# FIX: Change "out_dim / ratio" to "int(out_dim / ratio)"
# This forces the number of neurons to be an integer (e.g., 64 instead of 64.0)
content = content.replace("units=out_dim / ratio", "units=int(out_dim / ratio)")

with open("model.py", "w") as f:
    f.write(content)

print("✅ model.py has been patched to fix the Float/Integer error!")

✅ model.py has been patched to fix the Float/Integer error!


In [36]:
import os

# Read the current model.py (which already has the previous float/int fix)
with open("model.py", "r") as f:
    content = f.read()

# FIX: Change the signature to allow optional training argument
content = content.replace("def call(self, inputs, training):", "def call(self, inputs, training=None):")

# Save the fixed file
with open("model.py", "w") as f:
    f.write(content)

print("✅ model.py has been patched to fix the TransformerBlock error!")

✅ model.py has been patched to fix the TransformerBlock error!


In [37]:
import os

with open("model.py", "r") as f:
    content = f.read()

# FIX: In Keras 3, 'alpha' was renamed to 'negative_slope' for relu
# We replace the specific function call used in the MLP section
content = content.replace("tf.keras.activations.relu(x, alpha=0.1)", "tf.keras.activations.relu(x, negative_slope=0.1)")

# Save the fixed file
with open("model.py", "w") as f:
    f.write(content)

print("✅ model.py has been patched for Keras 3 compatibility!")

✅ model.py has been patched for Keras 3 compatibility!


In [38]:
!python train.py

2026-02-15 16:50:06.670861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771174206.702331   27358 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771174206.712805   27358 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771174206.736170   27358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771174206.736209   27358 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771174206.736217   27358 computation_placer.cc:177] computation placer alr

In [39]:
!python test.py

2026-02-15 17:23:49.660751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771176229.691300   37467 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771176229.701672   37467 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771176229.728215   37467 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771176229.728250   37467 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771176229.728258   37467 computation_placer.cc:177] computation placer alr